In [6]:
##############################################
# Programmer: Matthew Moore
# Class: CptS 322-01, Spring 2022
# Programming Assignment #4
# 3/3/2022
# 
# 
# Description: This program runs through all of the visualizations for the assignment. The purpose of the file is
# to test the predictive accuracy of all of the classification methods we build in the myclassifiers.py class file. 
##############################################
import importlib
import os
import numpy as np

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
import mysklearn.mysimplelinearregressor
from mysklearn.mysimplelinearregressor import MySimpleLinearRegressor
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MySimpleLinearRegressionClassifier,\
    MyKNeighborsClassifier,\
    MyDummyClassifier,\
    MyDecisionTreeClassifier

In [7]:
importlib.reload(np)
importlib.reload(os)
importlib.reload(mysklearn.myutils)
importlib.reload(mysklearn.mypytable)
importlib.reload(mysklearn.myclassifiers)
np.random.seed(0)

fname = os.path.join("input_data", "cbb.csv")
bball_table = MyPyTable()
bball_table.load_from_file(fname)

fname = os.path.join("input_data", "cbb2022.csv")
bball_table_test = MyPyTable()
bball_table_test.load_from_file(fname)
# stats_header = ['ADJOD','ADJDE','BARTHAG','EFG_O','EFG_D','TOR','TORD',\
    # 'ORB','DRB','FTR','FTRD','2P_O','2P_D','3P_O','3P_D','ADJ_T','WAB']

stats_header = ['ADJOE','ADJDE','BARTHAG','EFG_O','EFG_D','TOR','TORD',\
    'DRB','FTR','3P_O','3P_D','ADJ_T','WAB','BARTHAG','ADJOE','ADJDE','EFG_O','EFG_D','BARTHAG','SEED']

stats_cols = []
stats_cols_inner = []
stats_col = []
for stat in stats_header:
    stats_col.append(bball_table_test.get_column(stat))

for index in range(len(bball_table_test.data)):
    for stat_col in stats_col:
        stats_cols_inner.append(stat_col[index])
    stats_cols.append(stats_cols_inner)
    stats_cols_inner = []

X_test = stats_cols
stats_cols = []
stats_col = []
# Grabbing all the rows we want to use
for stat in stats_header:
    stats_col.append(bball_table.get_column(stat))

# Creating a new table with the rows based on the appropriate columns
for index in range(len(bball_table.data)):
    for stat_col in stats_col:
        stats_cols_inner.append(stat_col[index])
    stats_cols.append(stats_cols_inner)
    stats_cols_inner = []

X_train_bball = [stats for stats in stats_cols]
y_train_bball = [val for val in bball_table.get_column('POSTSEASON')]


In [8]:
importlib.reload(mysklearn.myutils)
importlib.reload(mysklearn.mypytable)
importlib.reload(mysklearn.myclassifiers)
importlib.reload(mysklearn.mysimplelinearregressor)
k_values = [1,2,3,4,5,6,7,8,9,10,11,12,13]
print("===========================================")
print("STEP 2: Nearest Neighbor MPG Classifier")
print("===========================================")
results = ["Champions","2ND","F4","E8","S16","R32","R64","R68"]
parallel_predictions = []
for k in k_values:
    knn_classifier = MyKNeighborsClassifier(k)
    y_train_auto = bball_table.get_column('POSTSEASON')
    knn_classifier.fit(X_train_bball,y_train_bball)
    y_predictions = knn_classifier.predict(X_test)
    teams = bball_table_test.get_column("TEAM")
    num_correct = 0
    parallel_predictions.append(y_predictions)
    for result in results:
        # print(result)
        for i, test_instance in enumerate(X_test):
            # actual_mpgs.append([test_instance[bball_table.column_names.index('POSTSEASON')]])
            if result == y_predictions[i]:
                # print(teams[i],"places as ",y_predictions[i])
                pass

for j, team in enumerate(parallel_predictions[0]):
    print(teams[j],"places as ",end=' ')
    for i, y_predictions in enumerate(parallel_predictions):
        # actual_mpgs.append([test_instance[bball_table.column_names.index('POSTSEASON')]])
        print(results.index(parallel_predictions[i][j]),end=' ')
     
    print("")


STEP 2: Nearest Neighbor MPG Classifier
Gonzaga places as  3 3 1 3 3 3 3 3 3 3 3 3 3 
Kansas places as  2 3 3 3 2 2 3 3 4 4 4 2 2 
Baylor places as  5 1 1 1 4 5 5 5 5 5 5 5 5 
Arizona places as  3 3 2 2 2 2 2 2 2 3 3 3 3 
San Francisco places as  6 5 5 5 5 5 5 5 5 5 5 5 5 
Loyola Chicago places as  2 2 2 5 5 5 5 5 5 5 5 5 5 
Davidson places as  6 6 6 6 6 6 6 6 6 6 6 6 6 
Miami FL places as  6 6 6 6 6 6 6 6 6 5 6 5 5 
Virginia Tech places as  6 6 6 5 5 5 5 5 5 5 5 5 5 
Michigan places as  6 6 6 6 6 6 6 6 6 6 6 6 5 
Notre Dame places as  6 5 6 6 6 6 5 6 6 6 6 6 6 
Iowa St. places as  4 6 5 5 5 5 5 5 6 5 5 5 6 
Rutgers places as  6 6 6 6 6 6 6 6 6 6 6 6 6 
Indiana places as  5 2 2 5 5 5 5 5 5 5 6 6 6 
UAB places as  5 5 6 6 6 6 5 5 5 5 5 5 5 
Wyoming places as  6 5 6 6 6 6 6 6 6 6 6 6 6 
New Mexico St. places as  5 5 6 6 6 5 6 6 6 6 6 6 6 
Richmond places as  7 7 7 6 6 6 6 6 6 6 6 6 6 
Vermont places as  5 5 5 5 6 6 6 6 6 6 6 6 6 
South Dakota St. places as  6 6 6 6 6 6 6 6 6 6 6 6 6 
Cha

In [9]:
importlib.reload(mysklearn.myutils)
importlib.reload(mysklearn.mypytable)
importlib.reload(mysklearn.myclassifiers)
importlib.reload(mysklearn.mysimplelinearregressor)
# RQ5 (fake) iPhone purchases dataset
iphone_col_names = ["standing", "job_status", "credit_rating", "buys_iphone"]
X_train_iphone = [
    [1, 3, "fair"],
    [1, 3, "excellent"],
    [2, 3, "fair"],
    [2, 2, "fair"],
    [2, 1, "fair"],
    [2, 1, "excellent"],
    [2, 1, "excellent"],
    [1, 2, "fair"],
    [1, 1, "fair"],
    [2, 2, "fair"],
    [1, 2, "excellent"],
    [2, 2, "excellent"],
    [2, 3, "fair"],
    [2, 2, "excellent"],
    [2, 3, "fair"]
]
y_train_iphone = ["no","no","yes","yes","yes","no","yes","no","yes","yes","yes","yes","yes","no","yes"]
my_tree = MyDecisionTreeClassifier()
my_tree.fit(X_train_iphone,y_train_iphone)
print(my_tree.tree)

['Attribute', 'att0', ['Value', 1, ['Attribute', 'att1', ['Value', 1, ['Leaf', 'yes', 1, 5]], ['Value', 2, ['Attribute', 'att2', ['Value', 'excellent', ['Leaf', 'yes', 1, 2]], ['Value', 'fair', ['Leaf', 'no', 1, 2]]]], ['Value', 3, ['Leaf', 'no', 2, 5]]]], ['Value', 2, ['Attribute', 'att2', ['Value', 'excellent', ['Leaf', 'no', 4, 10]], ['Value', 'fair', ['Leaf', 'yes', 6, 10]]]]]
